## Community Detection

<img src='Images/blogs.png'>

Real networks often show a modular structure, where we can see groups of nodes with nodes from the same group densely connected and nodes from different groups rarely connected. Such densely connected groups of nodes are called communities, and the process to find the communities or to partition the nodes into communities in a given network is called **community detection**.

## Community Detection

* Community: densely connected group of nodes (cf., connected component).
* Community detection: partition the nodes into communities (cf., clustering).

However, besides this intuition of what a community looks like, there is no unanimous definition of it. And accordingly, there exist many methods to find communities where each method has its own definition of communities. 

### Modularity

* Measure the “quality” of a certain partition of nodes.
* Developed by Girvan and Newman (cited 40000+ times).
* Can be applied to temporal networks, weighted networks, multiplex networks, and so on.

Modularity is a measure for the “quality” of certain partition of the nodes in a network. Given a partition, modularity compares the number of edges within clusters to the expected number of edges by chance. The expected number of edges is determined by a null model - a random network specified by the user. Modularity was first developed by Girvan and Newman in their seminar work, and has been extended to temporal networks, weighted networks, multiplex networks, and so on.

* Given an undirected network with $N$ nodes and $M$ edges, let
    * $A$ be its adjacency matrix; 
    * $d_i$ be the degree of node $i$. 
    
* Assume the nodes are divided into $K$ communities, and $c_i$ is the community that node $i$ belongs to. The modularity score of this partition is defined as:
$$
Q = \frac{1}{2M} \sum_{i=1}^N \sum_{j=1}^N [A_{ij} - \gamma\frac{d_i d_j}{2M}] \delta(c_i, c_j),
$$
    * $\delta(c_i, c_j)$ is the delta function such that $\delta(c_i, c_j)=1$ if $c_i = c_j$ (i.e., nodes $i$ and $j$ are in the same community) and $\delta(c_i, c_j)=0$ otherwise. 
    * $\frac{d_i d_j}{2M}$ is the expected number of edges between nodes $i$ and $j$
    * $\gamma$ is a resolution parameter that controls the relative importance of the null model.

Therefore, the summation in the equation is actually taken over all pairs of nodes from the same communities. Noting that $\frac{d_i d_j}{2M}$ is the expected number of edges between nodes $i$ and $j$ in the configuration model,  we can see that a partition will achieve a high modularity score if there are many within-cluster edges compared to the configuration model. The term $\frac{1}{2M}$ in front of the sum is a normalization factor to make the score between 0 and 1. 

Importantly, $\gamma$ is a resolution parameter that controls the relative importance of the null model. When it is zero, the optimal partition would be the one where all nodes are in one single community (which yields $Q=1$), while when it is positive infinite, the optimal partition would be the one where every node forms a community by its own. Therefore, by tuning the resolution parameter $\gamma$, it is possible to find communities at all scales. 

Modularity enables us to quantify the performance of different partitions, and hence, we can use it to find the optimal partition. The best partition, as described above, will have many edges within each community and few edges between communities, agreeing with our intuition on clusters or modules. Consider a hypothetical network below. 

#### Modularity Optimization

Find the partition that optimizes modularity
* Naive way: sweep through all possible partitions (how many are there for $N$ nodes?)
* Iterative method for local maximum (Louvain method)
    * Initially, each node is assigned to a community by itself.
    * In every step, each node is moved to the community such that modularity improves the most with this move. When no node can be reassigned, each community is aggregated into a node, and the process restarts again with the merged communities as new nodes. 
    * The process stops when there is only a single node left or when the modularity cannot be improved.

The most straightforward way to find the optimal partition is to sweep through all partitions and identify the one with the largest modularity. However, this is not feasible for networks of even moderate size because the number of possible partitions is explosive. Therefore, there are many heuristic approaches to a reasonably good partition, that is, a partition close to but not guaranteed to be the optimal. Those methods are usually iterative, greedy algorithms which successively improve the partition at each step. One particular method commonly used is called the Louvain method developed by Blondel *et al*. Specifically, the method proceeds as the following:

### Infomap

* Measure the “quality” of a certain partition of nodes from an information theoretic perspective.
* Developed by Rosvall and Bergstrom.
* Can be applied to temporal networks, weighted networks, multiplex networks, and so on.

Infomap is an information theoretic approach to the detection of communities. Roughly speaking, a group of nodes among which information flows quickly and easily are identified as a community, and the links between communities capture the avenues of information flow between those groups.

* Given a network and a partition of the nodes
* Consider a random walk on the network
<img src='Images/infomap_local.png' width='400'>
    * Blue,01,1,Yellow,0,110,01
* The description length of this path is the number of bits needed to encode this path
* This length will change if using a different coding scheme

Mathematically,  the average shortest description length of a single step of the random walks on a network is given by
$$
L(M) = q H(Q) + \sum_{i=1}^m p^i H(P^i),
$$
* $M$ is a given partition of the network.
* $q$ is the probability that the random walk switches communities on any given step 
* $H(Q)$ is the entropy of the community names
* $H(P^i)$ is the entropy of the within-community movements
* $p^i$ is the fraction of within-module movements that occur in community $i$, plus the probability of exiting community $i$ 

#### Description Length Optimization

Find the parition that optimize $L(M)$

* First, compute the fraction of time each node is visited by a random walker using the power method. 
* Using the visit frequencies calculated above, explore the space of possible partitions via a deterministic greedy search algorithm. 
* Refine the results with a simulated annealing approach using the heat-bath algorithm.

Similar to modularity, description length can be a measure of the quality of a partition, and hence we can use it to find the optimal partition. This is done by a heuristic algorithm developed by Rosvall and Bergstrom, which proceeds as the following:

#### Analogy in Geography

* Given a map of a region
    * Name the streets
    * Divide the region into cities
* So that it is easy to describe a route through the region
* Tradeoff
    * Reuse of short street names
    * Frequent transition between cities

Infomap uses a random walk on a network as a proxy for the information flow on the network, and aims to find a partition of the nodes such that the description length of random walks on the network is minimized. As an analogy in geography, think of each community as a city and the nodes as streets. We allow nodes in different communities to have same names just as we reuse the same street names in different cities. Then a random walk can be described by enumerating all the city names and street names visited by the random walk. For example, to go to my favorite restaurant, I would take the following path: 

Durham (city name) -> MLK (street name) -> 157 (street name) -> Chapel Hill (city name) -> main (street name). 

This description has a certain length and this length can change if the city and street names change. You would want popular streets (nodes) to have shorter names and reuse those names as much as possible. It might seem beneficial to have many communities so that you can reuse the street (node) names to a large extent, but then the city names will appear frequently in a typical path and that might increase the description length. Hence, the community detection problem is equivalent to solving an optimal coding problem. 

### Edge Betweenness

* Edges with high betweenness are usually bridges between densely connected groups of nodes.
* Find those groups (or communities) by identifying the high-betweenness edges.
* Developed by Newman and Girvan in their paper “Finding and evaluating community structure in networks” (2004).
<img src='Images/betweenness.png'>

As a centrality measure, edge betweenness can also be used to partition nodes in a network. The method was first developed by Newman and Girvan in their paper “Finding and evaluating community structure in networks” (2004).

Edges that sit on the many shortest paths in a network have high betweenness. Such edges are usually bridges between densely connected groups of nodes. Hence, we can find those groups (or communities) by identifying the high-betweenness edges.

Given a network with $N$ nodes and $M$ edges, the method will iteratively find and remove the edge with the largest betweenness. Specifically, the method proceeds as the following:

* Initially, all the nodes are in one single community.
* Calculate the betweenness for all the edges in the network.
* Find the edge with the largest betweenness and remove it from the network. This might divide the community to which the edge belongs into two communities.
* Compute the modularity of the current partition on the **original** network, and save the modularity score and the current partition. 
* Repeat all the steps above except for the initialization step, until the network cannot be divided further, i.e., every node is in a different community by its own.
* Finally, pick the partition with the highest modularity among all the partitions we have computed.

Note that the method uses modularity as a performance measure; do not confuse it with the modularity community detection method. Also, we compute modularity on the original network with all edges, not on the current working network where the edges with largest betweenness are removed.

* Generate a hierarchy of communities
<img src='Images/dendrogram.png'>
* Flexible regarding to when to stop
    * terminate the process when a certain number of communities are found
    * use other partition quality measures such as Infomap

The algorithm actually creates a hierarchy of communities, as it keeps dividing communities into sub-communities. For example, the first removal of an edge divides the nodes into two communities, the second removal divides one of the two communities into two sub-communities, and so on. This hierarchy can be represented as a dendrogram as illustrated below. Hence, the algorithm is in fact flexible regarding to when to stop: you can terminate the process when you find as many communities as you consider appropriate. Also, you can use other partition quality measures such as the description length from Infomap.

### Stochastic Block Model

* A generative, probability model for networks
* Assume the probability that two nodes will form an edge depends on the community memberships of the two nodes. 

Specifically, the adjacency matrix $A$ is modeled as a random matrix whose entries are Bernoulli variables with parameters $p_{kl}$:
$$
P(A_{ij}=1) = p_{C_i, C_j}, \forall i \neq j.
$$

If the within-community probabilities $p_{kk}$ are significantly larger than the between-community probabilities $p_{kl}, k \neq l$, then

If we arrange the rows and columns of the matrix in such  a way that nodes from the same community take consecutive rows and columns, and if the within-community probabilities $p_{kk}$ are significantly larger than the between-community probabilities $p_{kl}, k \neq l$, then the generated adjacency matrix will have dense blocks along the diagonal where each block corresponds to a community. See the simulated network and its adjacency matrix below. This pattern shows why the model is called a "block"" model.

<table>
    <tr>
        <td><img src='Images/sbm_matrix.png' width='400'></td>
        <td><img src='Images/sbm_network.png'></td>
    </tr>
</table>

Given a network with $N$ nodes and $M$ edges
* find the partition of the nodes  
* probablities $p_{kl}$ 
that maximize the likelihood of generating the observed network. 

This optimization problem is commonly addressed by the variational expectation maximization method developed by Mariadassou *et al*, or the spectral clustering method.

* Unrealistic assumption: all the nodes in the same community are equally likely to form edges
* Reality: very heterogeneous degree distributions
* Solution: degree-corrected stochastic block model
$$
P(A_{ij}=1) = r_i r_j p_{C_i, C_j}, \forall i \neq j.
$$
    * $r_i$ captures the connectivity of node $i$

A real problematic assumption of the stochastic block model is that nodes in the same community are equally likely to form edges. Most real networks have very heterogeneous degree distributions with a few nodes much more connected than others. To address this limit, a degree-corrected stochastic block model was developed by Karrer and Newman that takes into consideration the heterogeneous connectivity for different nodes. Specifically, it adds into the model another group of parameters $r_i, i=1,\cdots,N$, where $r_i$ captures the connectivity of node $i$.  And then the probability that two nodes will form an edge is defined as 
$$
P(A_{ij}=1) = r_i r_j p_{C_i, C_j}, \forall i \neq j.
$$

Given so many community detection methods, which one to use? 

Even for a particular method, there could be tuning parameters and what parameter values are appropriate?

### Consensus Clustering

* Combine different methods or different variants of one method to find a "consensus" partition of the network
* If there is a community structure, it should be magnified by the consensus between different methods
* Orginated from computer science, applied to community detection by Lancichinetti and Fortunato (2012)

Consensus clustering is a method to combine different methods or different variants of one single method to find a "consensus" partition of the network. In other words, it is not community detection method by itself, but it is a framework to combine existing methods to find communities. If the given network has even a weak community structure, and each method can detect some part of it, then the structure should be magnified by the consensus between those methods. It is not a new idea in clustering problems in computer science, but it was first applied to community detection in networks by Lancichinetti and Fortunato (2012).

Assume we have $K$ community detection methods of interest, denoted by $f_1, f_2, \cdots, f_K$. Given a network with $N$ nodes and $M$ edges, the consensus clustering algorithm proceeds as the following:

* For each method $f_i$, apply it to the network and obtain a partition $C^i$, where $C^i$ is a vector and $C^i_u$ is the community that node $u$ belongs to.    
* Construct a contingency network with the same nodes as  those in the original network but different edges. An edge exists between nodes $u$ and $v$ in the contingency network if $u$ and $v$ belong to the same community in any of the partitions $C^i$ (i.e., $C^i_u = C^i_v$, for some $i$), and this edge is weighted by how many times they are clustered together (i.e., $\sum_{i=1}^K {1}(C^i_u = C^i_v)$).
* Remove edges in the contingency network whose weights are below a certain threshold.
* Replace the original network by the contingency network and repeat all the steps above, until the partitions from different detection methods are the same.

Each $f_i$ should take a weighted network as input and output a partition of the nodes. If any $f_i$ cannot handle weighted network, then the contingency network should be binarized first.

### Example: Factions in the Karate Club

The data was collected and studied by Zachary in his paper "An Information Flow Model for Conflict and Fission in Small Groups" published in 1977. It was collected during a period of three years from 1970 to 1972. The karate club had 34 members, and hence the network consists of 34 nodes with each representing a member, and an edge exists between two nodes if they interacted outside the club. There are 78 undirected, unweighted edges in total. 

What makes this network an ideal example for community detection is an incident occurred during the study. 
The administrator of the club had a conflict with the instructor during the study and the club split into two in the end. About half of the members formed a new club with the instructor, and another half stayed with the administrator and found a new instructor. Therefore, there were obviously  two factions in the network, led by the instructor and the administrator respectively, long before the split, which provides a benchmark to test any community detection method.

First, load the network data into igraph as for the Florentine Family data:

In [3]:
library('igraph')
nodes = read.csv('dataset-karate-1977-nodes.csv')
edges = read.csv('dataset-karate-1977-edges.csv')
G = graph_from_data_frame(d=edges, vertices=nodes, directed=F)


Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union



The attribute we need is the faction ID each member actually joined after the split, which is called (no surprise) "faction".

Most community detection methods are implemented in igraph except for the stochastic block model and consensus clustering. But to perform modularity optimization with the resolution parameter, we need to install the package "resolution":

In [4]:
#install.packages("devtools")
#devtools::install_github("analyxcompany/resolution")
library(resolution)

> Side Note: if there is an error saying "devtool" not available for the R version ..., it is because the R version is too old. Updating R would help. Looking at the source page for devtool at https://cran.r-project.org/web/packages/devtools/index.html, version 3.0.2 and later are requried. 

Modularity

In [5]:
community = cluster_resolution(G, 1.5)
nodes$modularity = membership(community)

Infomap

In [11]:
community = cluster_infomap(G)
nodes$infomap = membership(community)

Betweenness

In [10]:
community = cluster_edge_betweenness(G)
nodes$between = membership(community)

Stochastic Block Model

In [15]:
library(randnet)
ssc = reg.SSP(as_adjacency_matrix(G, sparse=FALSE),K=2)
nodes$sbm = ssc$cluster

> Side note: again, if there is an error installing randnet like "not available for R", it is usually because that the R version is too old. Updating R should help. In general, there are many reasons for which this error could happen, see https://stackoverflow.com/questions/25721884/how-should-i-deal-with-package-xxx-is-not-available-for-r-version-x-y-z-wa for a good list.

### Excercise: Consensus Clustering

There is no package for consensus clustering, but it is not hard to write one. Here we only consider two metheds: modulairy and infomap.

In [16]:
G0 = G
detection_methods=c(cluster_louvain, cluster_infomap)
repeat{
  A=matrix(0, nrow = vcount(G), ncol = vcount(G))
  for (method in detection_methods) {
    community = method(G)
    for (k in 1:length(community)) {
      if (length(community[[k]])==1) next
      for (i in 1:(length(community[[k]])-1)) {
        u=community[[k]][i]
        for (j in (i+1):length(community[[k]])) {
          v=community[[k]][j]
          A[strtoi(u),strtoi(v)]=A[strtoi(u),strtoi(v)]+1}}}}
  a = unique(A[A>0])
  if (length(a)==1 && a==length(detection_methods)) break
  G = graph_from_adjacency_matrix(A, mode='upper', weighted=TRUE)}

nodes$consensus = membership(community)

### Excercise: Compare the partitions to the actual split of the club.

In [23]:
table(nodes$faction,nodes$modularity)

   
     1  2
  1  0 16
  2 17  1

In [ ]:
plot(community, G)

plot(G, vertex.color=nodes$modularity, vertex.label=nodes$faction)

* Modularity: clustering result is wrong for one node (#10).
* Infomap: correctly identifies the actual split of the club members except for one node (#10), but further divides one faction into two communities.
* Betweenness: correctly identifies the actual split of the club members except for one node (#3), but further divides one faction into two communities and the other into three communities.
* Stochastic Block Model: correctly identifies the actual split of the club members.
* Consensus Clustering: correctly identifies the actual split of the club members except for one node (#10), but further divides one faction into two communities.